## Prerequisite: Generate OpenAI key

Generate key from: https://platform.openai.com/api-keys

In [ ]:
!pip install -q langchain==0.2.2
!pip install -q langchain-openai==0.1.8

In [ ]:
import os

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.globals import set_debug
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
set_debug(False) # disable langchain debug mode

## Building Information Extraction AI

In [ ]:
mock_email = """
Dear Hiring Team,

    My name is Kasikorn Rakthai, and I am excited to apply for the Frontend Developer position at KBTG. With a Bachelor’s degree in Computer Science from Chulalongkorn University and over three years of experience in frontend development, I am confident in my ability to contribute to your innovative team.
I am proficient in HTML, CSS, JavaScript, and frameworks such as React.js and Angular. I successfully led a website redesign that increased user engagement by 30% and improved conversion rates by 20%.
KBTG's commitment to technological innovation greatly excites me, and I am eager to bring my skills to your team. Attached is my resume for your review. I am available for an interview at your convenience and can be reached at 089-123456.
Thank you for considering my application.

Best regards,
Kasikorn Rakthai
"""

In [ ]:
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a Hiring Manager at KBTG reading a candidate's email.
            A candidate's email text will be given.
            Your task is to identify the following information and convert it into JSON object with these key:
            - job_position: string, the job position which the candidate is applying for.
            - company_name: string, the company name which the candidate is applying for.
            - candidate_name: string, the name and surname of the candidate.
            Answer in the form of JSON object.
            """
        ),
        (
            "user", "{email}"
        )
    ]
)

parser = JsonOutputParser()
screen_chain = prompt_template | model | parser

screen_chain.invoke({"email": mock_email})

## Building decision logic

In [ ]:
def clean(text):
    return text.lower().strip().replace(" ", "")

opening_positions = [
    "Frontend Developer",
    "Backend Developer",
    "AI Developer",
    "Blockchain Developer"
]
opening_positions = set([clean(e) for e in opening_positions])

def make_decision(candidate_data):
    criteria_1 = clean(candidate_data["job_position"]) in opening_positions
    criteria_2 = clean(candidate_data["company_name"]) == clean("KBTG")

    return criteria_1 and criteria_2

def screen_email(email):
    candidate_data = screen_chain.invoke({"email": email})
    is_pass = make_decision(candidate_data)
    if is_pass:
        return "pass"

    return "failed"

In [ ]:
fe_email = mock_email
screen_email(fe_email)

In [ ]:
chef_email = """
Dear Hiring Manager,

    I am writing to express my interest in the Head Chef position at KBTG. With over 15 years of culinary experience, including 5 years as Executive Chef at renowned restaurants, I am confident in my ability to contribute to your esteemed establishment.
My expertise in diverse cuisines, team leadership, and innovative menu development align well with the vision of KBTG. I am excited about the opportunity to bring my passion for culinary excellence to your team.
Please find my resume attached for your review. I look forward to the possibility of discussing how my background, skills, and certifications can be an asset to KBTG.
Thank you for considering my application.

Sincerely,
Aroii Dee
"""

screen_email(chef_email)

## Complete response email draft by ChatGPT

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=1.0)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are Hiring Manager named "Khunthong Smith".
            Your task is to write an email response to candidate job application.
            """
        ),
        (
            "user", "Write the response email in formal English langauge which notify candidate named \"{candidate_name}\" that they \"{result}\""
        )
    ]
)

parser = StrOutputParser()
email_generation_chain = prompt_template | model | parser

def screen_email(email):
    candidate_data = screen_chain.invoke({"email": email})
    is_pass = make_decision(candidate_data)

    return email_generation_chain.invoke({"candidate_name": candidate_data, "result": "pass" if is_pass else "not pass"})

In [ ]:
screen_email(fe_email)

In [ ]:
screen_email(chef_email)

## Add support for multi language

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=1.0)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are Professional Translator.
            Your task is to identify the original langauge and translate content into standard English language.
            answer as JSON object with the following keys:
            - translated_content: string, a content translated in to standard English.
            - original_langauge_name: string, a language name of the given content.
            """
        ),
        (
            "user",
            "{email}"
        )
    ]
)

parser = JsonOutputParser()
input_translator_chain = prompt_template | model | parser

In [ ]:
th_email = """
เรียน ผู้จัดการฝ่ายบุคคล,

    ข้าพเจ้าเขียนจดหมายนี้เพื่อแสดงความสนใจในการสมัครตำแหน่งหัวหน้าเชฟที่บริษัท KBTG ด้วยประสบการณ์การทำงานด้านการทำอาหารมากกว่า 15 ปี รวมถึงการเป็นเชฟบริหารในร้านอาหารชื่อดังมานานกว่า 5 ปี ข้าพเจ้าเชื่อมั่นในความสามารถที่จะนำพาความสำเร็จมาสู่สถานประกอบการของท่าน
ความเชี่ยวชาญของข้าพเจ้าในอาหารหลากหลายประเภท ความสามารถในการนำทีม และการพัฒนาเมนูที่นวัตกรรมตรงกับวิสัยทัศน์ของ KBTG ข้าพเจ้ารู้สึกตื่นเต้นกับโอกาสที่จะนำความหลงใหลในการทำอาหารที่ยอดเยี่ยมมาสู่ทีมของท่าน
กรุณาพิจารณาประวัติการทำงานที่แนบมานี้ ข้าพเจ้าหวังเป็นอย่างยิ่งว่าจะได้พูดคุยเกี่ยวกับวิธีการที่ประสบการณ์ ทักษะ และใบรับรองต่างๆ ของข้าพเจ้าจะสามารถเป็นประโยชน์ต่อ KBTG
ขอขอบพระคุณที่พิจารณาการสมัครของข้าพเจ้า

ขอแสดงความนับถือ
อร่อย ดี
"""

input_translator_chain.invoke({"email": th_email})

In [ ]:
jp_email = """
採用担当者様

    この度、KBTGのヘッドシェフ職に応募させていただきたく、メールをお送りいたしました。15年以上の料理経験と、名高いレストランでの5年間のエグゼクティブシェフとしての経験を持つ私が、御社に貢献できると確信しております。
多様な料理の専門知識、チームリーダーシップ、革新的なメニュー開発のスキルは、KBTGのビジョンに合致していると考えております。料理の卓越性に対する私の情熱を御社のチームに活かせる機会を大変楽しみにしております。
添付した履歴書をご覧いただければ幸いです。私の経歴、スキル、および資格がKBTGにとってどのように役立つかを話し合う機会をいただけることを楽しみにしております。
ご検討の程、よろしくお願い申し上げます。

敬具
アロイ・ディー
"""

input_translator_chain.invoke({"email": jp_email})

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=1.0)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are Hiring Manager named "Khunthong Smith".
            Your task is to write an email response to candidate job application.
            """
        ),
        (
            "user", "Write the response email in formal {output_language} langauge which notify candidate named \"{candidate_name}\" that they \"{result}\""
        )
    ]
)

parser = StrOutputParser()
email_generation_chain = prompt_template | model | parser

def screen_email(email):
    translation = input_translator_chain.invoke({"email": email})
    eng_email_content = translation["translated_content"]

    candidate_data = screen_chain.invoke({"email": eng_email_content})
    is_pass = make_decision(candidate_data)

    return email_generation_chain.invoke({
        "candidate_name": candidate_data,
        "result": "pass" if is_pass else "not pass",
        "output_language": translation["original_language_name"]
    })

In [ ]:
screen_email(th_email)

In [ ]:
screen_email(jp_email)

In [ ]:
th_pass_email = """
    สวัสดีครับ ผมชื่อ กสิกร รักไทย ผมสนใจสมัครงานตำแหน่ง Frontend Developer ที่ KBTG ด้วยประสบการณ์ทำงานด้านการพัฒนาเว็บมากกว่า 3 ปี และปริญญาตรีสาขาวิทยาการคอมพิวเตอร์จากจุฬาลงกรณ์มหาวิทยาลัย ผมมั่นใจว่าสามารถนำทักษะของผมมาช่วยเสริมทีมของคุณได้อย่างแน่นอน
ผมมีความเชี่ยวชาญใน HTML, CSS, JavaScript และเฟรมเวิร์กต่างๆ เช่น React.js และ Angular ขณะทำงานที่ [บริษัทปัจจุบันหรือที่ทำงานเก่า] ผมได้เป็นผู้นำในการออกแบบเว็บไซต์ใหม่ที่ช่วยเพิ่มการมีส่วนร่วมของผู้ใช้ขึ้น 30% และอัตราการเปลี่ยนแปลงขึ้น 20%
ผมรู้สึกตื่นเต้นมากกับการที่ KBTG มุ่งมั่นที่จะใช้เทคโนโลยีล้ำสมัยเพื่อสร้างนวัตกรรมใหม่ ๆ ผมอยากจะนำทักษะของผมมาช่วยทีมของคุณให้ก้าวไปข้างหน้า เอกสารประวัติของผมแนบมาด้วยเพื่อให้คุณพิจารณา ผมพร้อมที่จะเข้าสัมภาษณ์ในเวลาที่คุณสะดวก สามารถติดต่อผมได้ที่ [เบอร์โทรศัพท์] หรือ [อีเมล]
ขอขอบคุณที่พิจารณาใบสมัครของผม

ด้วยความเคารพ
กสิกร รักไทย
"""

screen_email(th_pass_email)